In [3]:
import telegram
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import io
from read_db.CH import Getch
import pandas as pd
import pandahouse
import os

In [10]:
def full_report(chat=None):
    # данные бота и ID пользователя
    chat_id = chat or 223226505
    bot = telegram.Bot(token='5133536168:AAEeD7to2jwSY8hyjiwpD1eT2RIFFVC8NAM')

    # Запрашиваем в базе данные за прошедшую неделю и создаем колонку с короткой датой (понадобится для визуализации)
    query = """SELECT t1.Days, t1.Users_feed, t2.Users_mess, t3.Users_feed_with_mess, t4.Users_feed_without_mess, 
    t5.Views, t6.Likes, t7.Messages, t8.New_users_feed, t9.New_users_mess FROM

    (SELECT toDate(time) AS Days, COUNT(DISTINCT user_id) AS Users_feed
    FROM simulator_20220520.feed_actions
    WHERE toDate(time) BETWEEN today()-7 and today()-1
    GROUP BY toDate(time)) AS t1

    JOIN
    (SELECT toDate(time) AS Days, COUNT(DISTINCT user_id) AS Users_mess
    FROM simulator_20220520.message_actions
    WHERE toDate(time) BETWEEN today()-7 and today()-1
    GROUP BY toDate(time)) AS t2
    ON t1.Days=t2.Days

    JOIN 
    (SELECT toDate(time) AS Days, count(DISTINCT user_id) AS Users_feed_with_mess FROM
         (SELECT *
          FROM simulator_20220520.feed_actions) AS tt1
       JOIN
         (SELECT user_id
          FROM simulator_20220520.message_actions) AS tt1 USING user_id
    GROUP BY toDate(time)) AS t3
    ON t1.Days = t3.Days

    JOIN
    (SELECT toDate(time) AS Days, count(DISTINCT user_id) AS Users_feed_without_mess FROM
         (SELECT *
          FROM simulator_20220520.feed_actions) AS tt3
       ANTI JOIN
         (SELECT user_id
          FROM simulator_20220520.message_actions) AS tt4 USING user_id
    GROUP BY toDate(time)) AS t4
    ON t1.Days = t4.Days

    JOIN
    (SELECT toDate(time) AS Days, countIf(user_id, action='view') AS Views
    FROM simulator_20220520.feed_actions
    WHERE toDate(time) BETWEEN today()-7 and today()-1
    GROUP BY toDate(time)) AS t5
    ON t1.Days=t5.Days

    JOIN
    (SELECT toDate(time) AS Days, countIf(user_id, action='like') AS Likes
    FROM simulator_20220520.feed_actions
    WHERE toDate(time) BETWEEN today()-7 and today()-1
    GROUP BY toDate(time)) AS t6
    ON t1.Days=t6.Days

    JOIN
    (SELECT toDate(time) AS Days, COUNT(user_id) AS Messages
    FROM simulator_20220520.message_actions
    WHERE toDate(time) BETWEEN today()-7 and today()-1
    GROUP BY toDate(time)) AS t7
    ON t1.Days=t7.Days

    JOIN
    (SELECT toDate(first_occurence) AS Days,
           count(user_id) AS New_users_feed
    FROM
      (SELECT user_id,
              min(time) as first_occurence
       FROM simulator_20220520.feed_actions
       GROUP BY user_id) AS tt4
    GROUP BY toDate(first_occurence)) AS t8
    ON t1.Days=t8.Days

    JOIN
    (SELECT toDate(first_occurence) AS Days,
           count(user_id) AS New_users_mess
    FROM
      (SELECT user_id,
              min(time) as first_occurence
       FROM simulator_20220520.message_actions
       GROUP BY user_id) AS tt5
    GROUP BY toDate(first_occurence)) AS t9
    ON t1.Days=t9.Days 
    """

    df = Getch(query).df
    df['day'] = df['t1.Days'].dt.strftime('%d/%m')
    df = df.set_index('day')

    # Создаем переменные с метриками метриками
    dau_feed = df['t1.Users_feed']
    dau_mess = df['t2.Users_mess']
    dau_feed_with_mess = df['t3.Users_feed_with_mess']
    dau_feed_without_mess = df['t4.Users_feed_without_mess']
    views = df['t5.Views']
    likes = df['t6.Likes']
    messages = df['t7.Messages']
    new_users_feed = df['t8.New_users_feed']
    new_users_mess = df['t9.New_users_mess']
    
    # Строим 8 графиков с ключевыми метриками

    sns.set(rc={'figure.figsize':(16,18)})
    fig, axs = plt.subplots(nrows = 4, ncols=2)

    sns.lineplot(data=dau_feed, ax=axs[0, 0], color = 'g').set(ylabel = 'DAU (feed)', xlabel = None)
    sns.lineplot(data=dau_mess, ax=axs[0, 1], color = 'g').set(ylabel = 'DAU (messenger)', xlabel = None)
    sns.lineplot(data=dau_feed_with_mess, ax=axs[1, 0], color = 'r').set(ylabel = 'DAU (feed+messenger)', xlabel = None)
    sns.lineplot(data=dau_feed_without_mess, ax=axs[1, 1], color = 'r').set(ylabel = 'DAU (feed - messenger)', xlabel = None)
    sns.lineplot(data=[views, likes], ax=axs[2, 0]).set(ylabel = 'Views and likes', xlabel = None)
    sns.lineplot(data=messages, ax=axs[2, 1]).set(ylabel = 'Messages', xlabel = None)
    sns.lineplot(data=new_users_feed, ax=axs[3, 0], color = 'black').set(ylabel = 'New users (feed)', xlabel = None)
    sns.lineplot(data=new_users_mess, ax=axs[3, 1], color = 'black').set(ylabel = 'New users (messenger)', xlabel = None)

    plot_object = io.BytesIO()
    plt.savefig(plot_object)
    plot_object.seek(0)
    plot_object.name = 'weekly_metrics.png'
    plt.close()
    bot.sendPhoto(chat_id=chat_id, photo=plot_object)
    
    # Посылаем данные
    file_object = io.StringIO()
    df.to_csv(file_object)
    file_object.name = 'full_weekly_report.csv'
    file_object.seek(0)
    bot.sendDocument(chat_id=chat_id, document=file_object)

In [11]:
try:
    full_report()
except Exception as e:
    print(e)